In [9]:
import sys
sys.path.insert(1, '../library')
import database_helper
import pickle
import pandas as pd
import json
import numpy as np
import requests

In [10]:
start_time = '2024-09-04T12:00:00Z'

In [11]:
with open('selected_columns.pkl', 'rb') as f:
    selected_columns = pickle.load(f)

strings_to_exclude = ['icon', 'moon', 'warning']
selected_columns = [item for item in selected_columns if not any(substring in item for substring in strings_to_exclude)]

strings_to_include = ['days_0', '10838']
selected_columns = [item for item in selected_columns if all(substring in item for substring in strings_to_include)]

print(str(len(selected_columns)) + ' Features selected')
df = database_helper.query_data(field_list=selected_columns, start_time='2024-09-04T12:00:00Z')

9 Features selected
Executing Query 1 from 2024-09-04 12:00:00 to 2024-09-06 14:30:09
(18179, 17)


In [12]:
df.set_index('_time', inplace=True)
df = df.select_dtypes(include='float64')
df.interpolate(inplace=True)
df = df.resample('h').mean()
df.reset_index(inplace=True)
date_time = pd.to_datetime(df.pop('_time'), format='%d.%m.%Y %H:%M:%S')

display(df.head())
display(date_time[0:5])

,10838_days_0_precipitation,10838_days_0_sunrise,10838_days_0_sunset,10838_days_0_sunshine,10838_days_0_temperatureMax,10838_days_0_temperatureMin,10838_days_0_windDirection,10838_days_0_windGust,10838_days_0_windSpeed
0,48.000000,1.725425e+12,1.725473e+12,2640.0,239.0,160.0,2580.0,241.000000,74.0
1,48.000000,1.725425e+12,1.725473e+12,2640.0,239.0,160.0,2580.0,241.000000,74.0
2,48.000000,1.725425e+12,1.725473e+12,2640.0,239.0,160.0,2580.0,241.000000,74.0
3,48.000000,1.725425e+12,1.725473e+12,2640.0,239.0,160.0,2580.0,241.000000,74.0
4,46.333333,1.725425e+12,1.725473e+12,2652.5,239.0,160.0,2580.0,233.083333,74.0


0   2024-09-04 12:00:00+00:00
1   2024-09-04 13:00:00+00:00
2   2024-09-04 14:00:00+00:00
3   2024-09-04 15:00:00+00:00
4   2024-09-04 16:00:00+00:00
Name: _time, dtype: datetime64[ns, UTC]

In [13]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [14]:
df = (df - df.mean()) / df.std()

In [15]:

num_features = df.shape[1]
time_range = 24
df = df[:time_range]
print(df.shape, num_features, time_range)
df.reset_index(drop=True, inplace=True)
display(df.head())
#input_data = np.reshape(df, ( 1, time_range, num_features))
data = json.dumps({"signature_name": "serving_default", "instances": np.array(df[0:1]).tolist()}) #
print(data)



(24, 13) 13 24


,10838_days_0_precipitation,10838_days_0_sunrise,10838_days_0_sunset,10838_days_0_sunshine,10838_days_0_temperatureMax,10838_days_0_temperatureMin,10838_days_0_windDirection,10838_days_0_windGust,10838_days_0_windSpeed,Day sin,Day cos,Year sin,Year cos
0,1.116636,-1.575841,-1.575836,-1.446962,-0.661165,0.053917,1.838029,-0.319066,-0.847306,0.021340,-1.296701,1.702558,-1.677029
1,1.116636,-1.575841,-1.575836,-1.446962,-0.661165,0.053917,1.838029,-0.319066,-0.847306,-0.349876,-1.249920,1.632783,-1.610323
2,1.116636,-1.575841,-1.575836,-1.446962,-0.661165,0.053917,1.838029,-0.319066,-0.847306,-0.695795,-1.112762,1.563110,-1.543595
3,1.116636,-1.575841,-1.575836,-1.446962,-0.661165,0.053917,1.838029,-0.319066,-0.847306,-0.992842,-0.894577,1.493539,-1.476843
4,0.996211,-1.575841,-1.575836,-1.427125,-0.661165,0.053917,1.838029,-0.450270,-0.847306,-1.220774,-0.610232,1.424070,-1.410068


{"signature_name": "serving_default", "instances": [[1.1166356002902682, -1.5758410240074838, -1.575836090522189, -1.4469615648837386, -0.6611651140147834, 0.05391666008949309, 1.8380293984787213, -0.319066184178734, -0.8473056536129032, 0.021340225734663605, -1.2967013759146937, 1.7025584095396318, -1.6770286303122126]]}


In [16]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/waether-timeseries-forecasts:predict', data=data, headers=headers)
response = json.loads(json_response.text)
print(response)
predictions = ['predictions']



{'predictions': [[0.143929884, -1.45822561, -1.44466472, -1.00462425, -1.02641904, 0.617316, 0.432807058, -0.180816293, 0.101159424, -0.0148573825, -0.992014408, 1.39010584, -1.39543092]]}
